In [1]:
!pip install seqeval

In [2]:
import pandas as pd
import numpy as np

import os

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# verify GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Convert to DataFrame for EDA

In [5]:
def convert_data(filepath):

  # Read the data from the text file
  with open(filepath, "r") as file:
      lines = file.readlines()

  # Define an empty list to store the data
  data = []

  # Iterate over each line in the file
  for line in lines:
      # Split the line by spaces
      parts = line.strip().split()

      # Check if the line has the expected number of elements
      if len(parts) == 9:
          # Extract the values from the line
          text_file_name = parts[0]
          sentence_line_number = int(parts[1])
          sentence_word_index = int(parts[2])
          sentence_seq = parts[3]
          start_token = int(parts[4])
          end_token = int(parts[5])
          original_word = parts[6]
          word = parts[7]
          label = parts[8]

          # Append the values as a tuple to the data list
          data.append((text_file_name, sentence_line_number, sentence_word_index, sentence_seq,
                      start_token, end_token, original_word, word, label))

  # Create a DataFrame from the data list with appropriate column names
  df = pd.DataFrame(data, columns=['text_file_name', 'sentence_line_number', 'sentence_word_index',
                                  'sentence_seq', 'start_token', 'end_token', 'original_word',
                                  'word', 'label'])

  return df


In [6]:
train_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_train.txt"
test_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_test.txt"

train = convert_data(train_data_path)
test = convert_data(test_data_path)

print(f"Length of train: {len(train)}")
print(f"Length of test: {len(test)}")


Length of train: 895141
Length of test: 585761


In [7]:
df = train.copy()
df['sentence_line_number'].nunique()

1053

In [8]:
df['label'].value_counts().sort_values()

label
I-Route           397
B-Duration        592
I-ADE             776
B-ADE             956
I-Duration       1034
I-Reason         3125
B-Reason         3791
I-Form           4173
B-Dosage         4221
I-Drug           4298
B-Route          5475
B-Frequency      6279
I-Strength       6617
B-Form           6647
B-Strength       6691
I-Dosage         8779
I-Frequency     13023
B-Drug          16222
O              802045
Name: count, dtype: int64

### Label labels

In [10]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in df['label'].values.tolist()]

# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

print(unique_labels)

{'B-Form', 'O', 'I-Drug', 'B-Strength', 'I-Dosage', 'I-Frequency', 'B-Dosage', 'B-ADE', 'B-Frequency', 'B-Route', 'I-Duration', 'I-ADE', 'I-Reason', 'I-Route', 'B-Drug', 'I-Form', 'B-Reason', 'I-Strength', 'B-Duration'}


In [11]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'B-ADE': 0, 'B-Dosage': 1, 'B-Drug': 2, 'B-Duration': 3, 'B-Form': 4, 'B-Frequency': 5, 'B-Reason': 6, 'B-Route': 7, 'B-Strength': 8, 'I-ADE': 9, 'I-Dosage': 10, 'I-Drug': 11, 'I-Duration': 12, 'I-Form': 13, 'I-Frequency': 14, 'I-Reason': 15, 'I-Route': 16, 'I-Strength': 17, 'O': 18}


In [14]:
def formatted_df(df):
  df['sentence'] = df[[
      'text_file_name',
      'sentence_line_number',
      'original_word',
      'label']].groupby(
          ['text_file_name', 'sentence_line_number'])['original_word'].transform(lambda x: ' '.join(x))

  df['word_labels'] = df[[
      'text_file_name',
      'sentence_line_number',
      'original_word',
      'label']].groupby(
          ['text_file_name', 'sentence_line_number'])['label'].transform(lambda x: ','.join(x))

  return df

In [15]:
df = formatted_df(df)
df_test = formatted_df(test)

In [16]:
def sentence_level_data_fn(df):
  sentence_level_data = df[["text_file_name", "sentence_line_number", "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
  return sentence_level_data

sentence_level_train = sentence_level_data_fn(df)
sentence_level_test = sentence_level_data_fn(df_test)

In [17]:
sentence_level_data = sentence_level_train[["text_file_name", "sentence_line_number", "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
sentence_level_test = sentence_level_test[["text_file_name","sentence_line_number",  "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)

sentence_level_data.head()

,text_file_name,sentence_line_number,sentence,word_labels
0,data/training_20180910/110727.txt,1,Admission Date : [ * * 2202 - 1 - 8 * * ] Disc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,data/training_20180910/110727.txt,3,Date of Birth : [ * * 2163 - 9 - 18 * * ] Sex : M,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,data/training_20180910/110727.txt,5,Service : MEDICINE,"O,O,O"
3,data/training_20180910/110727.txt,7,Allergies :,"O,O"
4,data/training_20180910/110727.txt,8,Keflex / Orencia / Remicade,"B-Drug,O,B-Drug,O,B-Drug"


In [18]:
len(sentence_level_data)

83321

In [19]:
sentence_level_data.iloc[7020].sentence

'your chemistries .'

In [20]:
sentence_level_data.iloc[7020].word_labels

'O,O,O'

In [21]:
print(sentence_level_data.iloc[21].sentence)
print(sentence_level_data.iloc[21].word_labels)

fx on x - ray . This was thought to be a psoriatic arthritis flare ,
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


#### Some sentences are really short. *Try* concatenating sentence to get longer inputs

In [ ]:
df = sentence_level_data.sort_values(by=['text_file_name', 'sentence_line_number'])
df_test = sentence_level_test.sort_values(by=['text_file_name', 'sentence_line_number'])

def cs(dataframe):
    processed_data_list = []

    # Group by 'text_file_name'
    grouped = dataframe.groupby("text_file_name")

    for name, group in grouped:
        sentences = group['sentence'].tolist()
        # Labels are already strings, so we take them as is
        labels = group['word_labels'].tolist()

        for i in range(0, len(sentences), 5):
            end_index = min(i + 5, len(sentences))
            current_batch = sentences[i:end_index]
            current_labels = labels[i:end_index]

            concatenated_sentence = " ".join(current_batch)
            # Concatenate labels as they are, assuming they're correctly formatted strings
            concatenated_label = ",".join(current_labels)

            processed_data_list.append({
                "text_file_name": name,
                "sentences": concatenated_sentence,
                "labels": concatenated_label
            })

    processed_data = pd.DataFrame(processed_data_list)
    return processed_data


# Correcting the function to properly handle labels
concatenated_df = cs(df)
concatenated_test= cs(df_test)

concatenated_df



#### Comment this out if you want sentence level dataset.

In [24]:
data = concatenated_df.copy()
data_test = concatenated_test.copy()

data.rename(columns={'concatenated_sentence': 'sentence', 'concatenated_labels': 'word_labels'}, inplace=True)
data_test.rename(columns={'concatenated_sentence': 'sentence', 'concatenated_labels': 'word_labels'}, inplace=True)

### It looks reasonable, let's proceed with training

In [26]:
from transformers import BertTokenizerFast, BertForTokenClassification

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [27]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
TEST_BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')


In [28]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentences[index].strip().split()
        word_labels = self.data.labels[index].split(",")
        # sentence_id = self.data.iloc[index]['sentence_id']

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True,
                             padding='max_length',
                             is_split_into_words=True,
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        item['index'] = index

        return item

  def __len__(self):
        return self.len

In [29]:
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(data, test_size=0.2, shuffle=False)


In [30]:
train_dataset.reset_index(drop=True, inplace=True)
val_dataset.reset_index(drop=True, inplace=True)

In [31]:
train_dataset['sent_id']  = train_dataset.index
val_dataset['sent_id']= val_dataset.index
data_test['sent_id'] = data_test.index


In [32]:
print("FULL Dataet: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))
print("TEST Dataset: {}".format(data_test.shape))

data_test.reset_index(drop=True, inplace=True)

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
val_set = dataset(val_dataset, tokenizer, MAX_LEN)
test_set = dataset(data_test, tokenizer, MAX_LEN)

FULL Dataet: (16788, 3)
TRAIN Dataset: (13430, 4)
VAL Dataset: (3358, 4)
TEST Dataset: (11146, 4)


In [33]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[3]["input_ids"]), training_set[3]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
to          18
an          18
OS          18
##H         -100
with        18
d           18
##ys        -100
##p         -100
##nea       -100
now         18
admitted    18
to          18
the         18
MI          18
##CU        -100
after       18
P           18
##EA        -100
arrest      18
x           18
##2         -100
.           18
The         18
patient     18
initially   18
presented   18
to          18
L           18
##G         -100
##H         -100
E           18
##D         -100
with        18
h           18
##y         -100
##pox       -100
##em        -100
##ic        -100
respiratory  18
distress    18
.           18
While       18
at          18
the         18
OS          18
##H         -100
,           18
he          18
received    18
CT          2
##X         -100
,           18
a           2
##zi        -100
##th        -100
##rom       -100
##y         -100
##cin       -100
,           18
SC          7
e           2
##pine      -100
##ph        

In [34]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **test_params)


In [36]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(3.0790, device='cuda:0', grad_fn=<NllLossBackward0>)

In [37]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 19])

In [38]:
import torch.nn.functional as F

def focal_loss(logits, labels, alpha=0.25, gamma=2.0, ignore_index=-100):
    """
    logits: [batch_size, seq_len, num_labels] - model predictions
    labels: [batch_size, seq_len] - ground truth labels
    """
    # Calculate Cross Entropy Loss without reduction
    ce_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='none', ignore_index=ignore_index)

    # Get the predictions
    pred_probs = F.softmax(logits.view(-1, logits.size(-1)), dim=-1)
    pred_class = labels.view(-1)

    # focusing parameter
    gamma = gamma

    # Filter out 'ignore_index' labels
    filtered = labels.view(-1) != ignore_index

    # Calculate focal loss
    ce_loss_filtered = ce_loss[filtered]
    pred_probs_filtered = pred_probs[filtered]
    pred_class_filtered = pred_class[filtered]

    # Construct the loss
    pt = pred_probs_filtered.gather(1, pred_class_filtered.unsqueeze(-1)).squeeze()
    loss = ((1 - pt) ** gamma * ce_loss_filtered).mean()  # mean over the batch

    return loss



In [39]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [40]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []

    model.train()

    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)
    #     global_attention_mask = torch.zeros(
    # input_ids.shape, dtype=torch.long, device=input_ids.device

        # Perform a forward pass to get the logits
        outputs = model(input_ids=ids, attention_mask=mask)

        # Calculate the loss using focal_loss function directly
        loss = focal_loss(outputs.logits, labels)  # Use the focal_loss function here directly

        tr_loss += loss.item()
        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # Decoding the logits to compute accuracy
        active_logits = outputs.logits.view(-1, model.num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)

        # Ignoring the predictions of the padding tokens
        active_accuracy = labels.view(-1) != -100
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(active_labels, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels.cpu().numpy())
        tr_preds.extend(predictions.cpu().numpy())

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")


## Train

In [41]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.7574527263641357
Training loss per 100 training steps: 0.5802953314441855
Training loss per 100 training steps: 0.4069263437040039
Training loss per 100 training steps: 0.3151090971992484
Training loss per 100 training steps: 0.2632187738760021
Training loss per 100 training steps: 0.22767525625404922
Training loss per 100 training steps: 0.20408259959248926
Training loss per 100 training steps: 0.1875862858044206
Training loss per 100 training steps: 0.17112036843086678
Training loss per 100 training steps: 0.15974655264041326
Training loss per 100 training steps: 0.14992563368150286
Training loss per 100 training steps: 0.14109611066098485
Training loss per 100 training steps: 0.13381413058069316
Training loss per 100 training steps: 0.12726275831048936
Training loss per 100 training steps: 0.1214326484019709
Training loss per 100 training steps: 0.11663676413690295
Training loss per 100 training steps: 0.1127649205576345
Trai

## Validate

In [42]:
def valid(model, testing_loader):
    model.eval()
    eval_loss, eval_accuracy = 0, 0

    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, indices = [], [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)
            # indices_list = batch[idx]

            outputs = model(input_ids=ids, attention_mask=mask)
            loss = focal_loss(outputs.logits, labels)  # Use the same focal_loss function

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # Decoding the logits to compute accuracy
            active_logits = outputs.logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)

            # Ignoring the predictions of the padding tokens
            active_accuracy = labels.view(-1) != -100
            active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

            labels = torch.masked_select(active_labels, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels.cpu().numpy())
            eval_preds.extend(predictions.cpu().numpy())

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

            indices.extend(batch['index'].tolist())



    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return indices,labels, predictions



In [43]:
indices, labels, predictions = valid(model, val_loader)

Validation loss per 100 evaluation steps: 0.008838528767228127
Validation loss per 100 evaluation steps: 0.030501436968707184
Validation loss per 100 evaluation steps: 0.03166459382074792
Validation loss per 100 evaluation steps: 0.03030670038333332
Validation loss per 100 evaluation steps: 0.031043381571355003
Validation loss per 100 evaluation steps: 0.03252436430024292
Validation loss per 100 evaluation steps: 0.03245420324389236
Validation loss per 100 evaluation steps: 0.03226016557406876
Validation loss per 100 evaluation steps: 0.03299174324748809
Validation loss per 100 evaluation steps: 0.03214605947634137
Validation loss per 100 evaluation steps: 0.03280890245574968
Validation loss per 100 evaluation steps: 0.03247020354457121
Validation loss per 100 evaluation steps: 0.03249185332263836
Validation loss per 100 evaluation steps: 0.03275368067879558
Validation loss per 100 evaluation steps: 0.03287561882657808
Validation loss per 100 evaluation steps: 0.03216793327669484
Valid

In [44]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         ADE       0.38      0.37      0.37       165
      Dosage       0.84      0.89      0.87       782
        Drug       0.86      0.92      0.89      3216
    Duration       0.56      0.71      0.62       120
        Form       0.91      0.88      0.90      1269
   Frequency       0.77      0.83      0.80      1269
      Reason       0.46      0.68      0.55       698
       Route       0.91      0.92      0.92      1143
    Strength       0.93      0.93      0.93      1341

   micro avg       0.82      0.87      0.84     10003
   macro avg       0.74      0.79      0.76     10003
weighted avg       0.83      0.87      0.85     10003



## Evaluation

 - convert val set back to word level

In [45]:
val_dataset['label_count'] = val_dataset['labels'].str.split(',').apply(len)

def split_predictions(test_predictions, label_count):
    split_predictions = []
    start_idx = 0
    for count in label_count:
        end_idx = start_idx + count
        split_predictions.append(test_predictions[start_idx:end_idx])
        start_idx = end_idx
    return split_predictions

broken_up_val_preds = split_predictions(predictions, val_dataset['label_count'])

val_dataset['predictions'] = broken_up_val_preds

In [46]:
# indices

In [50]:
val_dataset['preds_idx'] = indices
val_dataset.to_csv('/content/drive/MyDrive/266_final/bertbasecase_validation_df.csv')

In [ ]:
val_dataset.head()


### Test Set

In [47]:
test_indices, test_labels, test_predictions = valid(model, test_loader)

Validation loss per 100 evaluation steps: 0.033017177134752274
Validation loss per 100 evaluation steps: 0.02767990004400363
Validation loss per 100 evaluation steps: 0.028100907274368984
Validation loss per 100 evaluation steps: 0.02900631089178084
Validation Loss: 0.02930186528653803
Validation Accuracy: 0.9793069561968034


In [48]:
from seqeval.metrics import classification_report

print(classification_report([test_labels], [test_predictions]))

              precision    recall  f1-score   support

         ADE       0.45      0.39      0.42       634
      Dosage       0.87      0.91      0.89      2704
        Drug       0.88      0.94      0.91     10594
    Duration       0.55      0.68      0.61       385
        Form       0.94      0.91      0.92      4373
   Frequency       0.78      0.82      0.80      4149
      Reason       0.47      0.64      0.54      2559
       Route       0.94      0.94      0.94      3513
    Strength       0.94      0.96      0.95      4237

   micro avg       0.84      0.88      0.86     33148
   macro avg       0.76      0.80      0.78     33148
weighted avg       0.85      0.88      0.86     33148



### Save

In [60]:
import os

# directory = "/content/drive/MyDrive/266_final/bertbasecase_final"
# tokenizer.save_vocabulary("/content/drive/MyDrive/266_final/bertbasecase_final")
# model.save_pretrained(directory, safe_serialization=False)

('/content/drive/MyDrive/266_final/bertbasecase_final/vocab.txt',)

Resources

### https://github.com/lcampillos/Medical-NER/blob/master/bert_ner.ipynb
### https://medium.com/analytics-vidhya/bio-tagged-text-to-original-text-99b05da6664